# Код с результатами

In [10]:
%%writefile script.py
# ============================================================
# SISSO for /content/dataset_new_amo.dat  (amo + charge)
# CV5 on TRAIN (80%) -> best n_term  |  Hold-out TEST (20%)
# Saves: amo_sisso_report.txt, amo_holdout_predictions.csv,
#        amo_rmse_comparison.png, amo_parity_validation.png
# ============================================================
from src.TorchSisso import SissoModel
import sys, subprocess, importlib, re, time, gc
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(1)
from sklearn.model_selection import train_test_split, KFold


# ---------- 1) Load AMO dataset ----------
DATA_PATH = Path("train.dat")
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Файл не найден: {DATA_PATH}")

# пробельный разделитель, игнорируем строки с '#'
new_amo_ch = pd.read_csv(DATA_PATH, delim_whitespace=True, comment="#")
print("Загружено:", new_amo_ch.shape)
#display(new_amo_ch.head(3))


# ---------- 2) Target rename + numeric-only ----------
new_amo_ch = new_amo_ch.rename(columns={'Energy_Adsorption': 'Eads'})
if "Eads" not in new_amo_ch.columns:
    raise ValueError("Не найден целевой столбец 'Eads'.")

num_cols = new_amo_ch.select_dtypes(include=["number"]).columns.tolist()
if "Eads" not in num_cols:
    new_amo_ch["Eads"] = pd.to_numeric(new_amo_ch["Eads"], errors="coerce")
    num_cols = new_amo_ch.select_dtypes(include=["number"]).columns.tolist()
    if "Eads" not in num_cols:
        raise ValueError("'Eads' не числовая и не приводится к числу.")

feature_cols = [c for c in num_cols if c != "Eads"]
df_amo_num = new_amo_ch[["Eads"] + feature_cols].copy().dropna().astype("float32")
print(f"Численных фич: {len(feature_cols)} | строк: {len(df_amo_num)}")
#display(df_amo_num.head(3))


# ---------- 3) Hyperparams ----------
FAST = False            # первый стабильный прогон; потом можно False
SEED = 42
FOLDS = 5
FOLD_TIMEOUT_SEC = 300

safe_ops = ['+','-','*','/','pow(2)','abs']   # устойчивые операторы

if FAST:
    operators   = safe_ops
    N_EXPANSION = 2         # Tier=2 (быстрее)
    K_SUBSPACE  = 8
    N_TERMS_GRID = (1, 2)   # перебор размерности дескриптора
else:
    operators   = safe_ops  # можно добавить 'sqrt','ln','exp' позже
    N_EXPANSION = 3
    K_SUBSPACE  = 10
    N_TERMS_GRID = (1, 2, 3)

USE_GPU = False


# ---------- 4) Metrics & utilities ----------
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    return float(np.sqrt(np.mean((y_pred - y_true) ** 2)))

def mae(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    return float(np.mean(np.abs(y_pred - y_true)))

def r2_score(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot

# парсинг строки уравнения в выражение (в этой версии нет sm.predict)
def _transform_equation(eq: str) -> str:
    expr = eq.strip()
    expr = re.sub(r'pow\(\s*2\s*\)\s*\(\s*([^)]+?)\s*\)', r'(\1)**2', expr)
    expr = re.sub(r'pow\(\s*3\s*\)\s*\(\s*([^)]+?)\s*\)', r'(\1)**3', expr)
    expr = re.sub(r'\^([23])', r'**\1', expr)
    return expr

def predict_from_equation(equation: str, X_df: pd.DataFrame) -> np.ndarray:
    expr = _transform_equation(equation)
    ns = {c: X_df[c].to_numpy() for c in X_df.columns}
    ns.update({'abs': np.abs, 'np': np})
    y = eval(expr, {"__builtins__": {}}, ns)
    return np.asarray(y, dtype=float).ravel()

# подстройщик конструктора для разных версий пакета
def make_sisso_model(df: pd.DataFrame, **kwargs):
    for key in ("dataframe","df","data","dataset"):
        try:
            return SissoModel(**{key: df}, **kwargs)
        except TypeError:
            pass
    return SissoModel(df, **kwargs)


# ---------- 5) Split: CV только на TRAIN (80%), TEST = 20% ----------
train_idx, val_idx = train_test_split(
    np.arange(len(df_amo_num)), test_size=0.20, random_state=SEED, shuffle=True
)
df_train = df_amo_num.iloc[train_idx].reset_index(drop=True)
df_val   = df_amo_num.iloc[val_idx].reset_index(drop=True)

class TimeoutException(Exception): pass

def cv_rmse_for_nterm_on_train(df_tr_all: pd.DataFrame, n_term: int, folds: int = 5, seed: int = 42):
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    ys = df_tr_all["Eads"].to_numpy()
    Xs = df_tr_all.drop(columns=["Eads"])
    cols = Xs.columns.tolist()
    Xs = Xs.to_numpy()
    rmses = []
    for i, (tr, te) in enumerate(kf.split(Xs), 1):
        df_tr = pd.DataFrame(np.column_stack([ys[tr], Xs[tr]]), columns=["Eads"] + cols)
        df_te = pd.DataFrame(np.column_stack([ys[te],  Xs[te]]), columns=["Eads"] + cols)

        try:
            sm = make_sisso_model(
                df_tr,
                operators=operators,
                n_expansion=N_EXPANSION,
                n_term=n_term,
                k=K_SUBSPACE,
                use_gpu=USE_GPU
            )
            rmse_tr, eq, r2_tr, _ = sm.fit()
            y_pred = predict_from_equation(eq, df_te.drop(columns=["Eads"]))
            rmses.append(rmse(df_te["Eads"].to_numpy(), y_pred))
        except TimeoutException:
            print(f"[CV{folds} n_term={n_term}] Fold {i} — TIMEOUT ({FOLD_TIMEOUT_SEC}s)")
            rmses.append(np.nan)
        finally:
            del sm; gc.collect()
    rmses = np.array(rmses, dtype=float)
    rmses = rmses[~np.isnan(rmses)]
    if len(rmses) == 0: return np.array([np.inf])
    return rmses

def main():
    # --- CV5 на TRAIN для выбора n_term ---
    cv_summary = []
    for n_term in N_TERMS_GRID:
        scores = cv_rmse_for_nterm_on_train(df_train, n_term, folds=FOLDS, seed=SEED)
        cv_summary.append((n_term, scores.mean(), scores.std(), scores))
        print(f"CV{FOLDS} (TRAIN) n_term={n_term}: RMSE {scores.mean():.5f} ± {scores.std():.5f} | folds={np.round(scores,5)}")

    cv_summary.sort(key=lambda x: x[1])
    best_n_term, cv_mean, cv_std, cv_all = cv_summary[0]
    print(f"\nЛучший n_term по CV{FOLDS} на TRAIN: {best_n_term} (mean RMSE={cv_mean:.5f} ± {cv_std:.5f})")


    # ---------- 6) Full-fit на ВСЕХ данных ----------
    sm_full = make_sisso_model(
        df_amo_num,
        operators=operators,
        n_expansion=N_EXPANSION,
        n_term=best_n_term,
        k=K_SUBSPACE,
        use_gpu=USE_GPU
    )
    rmse_full_fit, eq_full, r2_full_rep, _ = sm_full.fit()
    y_full_pred = predict_from_equation(eq_full, df_amo_num.drop(columns=["Eads"]))
    overall_rmse = rmse(df_amo_num["Eads"].to_numpy(), y_full_pred)
    overall_mae  = mae(df_amo_num["Eads"].to_numpy(), y_full_pred)
    overall_r2   = r2_score(df_amo_num["Eads"].to_numpy(), y_full_pred)

    print("\n=== Full-fit on ALL (AMO) ===")
    print("Equation:", eq_full)
    print(f"RMSE={overall_rmse:.5f} | MAE={overall_mae:.5f} | R²={overall_r2:.5f}")


    # ---------- 7) Hold-out TEST (20%) ----------
    sm_hold = make_sisso_model(
        df_train,
        operators=operators,
        n_expansion=N_EXPANSION,
        n_term=best_n_term,
        k=K_SUBSPACE,
        use_gpu=USE_GPU
    )
    rmse_tr_hold, eq_hold, r2_tr_hold, _ = sm_hold.fit()

    X_val = df_val.drop(columns=["Eads"])
    y_val_true = df_val["Eads"].to_numpy()
    y_val_pred = predict_from_equation(eq_hold, X_val)

    val_rmse = rmse(y_val_true, y_val_pred)
    val_mae  = mae(y_val_true, y_val_pred)
    val_r2   = r2_score(y_val_true, y_val_pred)

    print("\n=== Hold-out (20%) — AMO ===")
    print("Equation (trained on 80%):", eq_hold)
    print(f"Train RMSE (80%)={rmse_tr_hold:.5f}")
    print(f"Val  RMSE (20%)={val_rmse:.5f} | MAE={val_mae:.5f} | R²={val_r2:.5f}")


    # # ---------- 8) Save predictions & plots ----------
    # # CSV с тестовыми предсказаниями
    # holdout_df = pd.DataFrame({
    #     "Eads_true": y_val_true,
    #     "Eads_pred": y_val_pred,
    # })
    # holdout_df["residual"]  = holdout_df["Eads_pred"] - holdout_df["Eads_true"]
    # holdout_df["abs_error"] = np.abs(holdout_df["residual"])
    # holdout_df.to_csv("amo_holdout_predictions.csv", index=False)
    # print("Saved: amo_holdout_predictions.csv")

    # # RMSE bar
    # labels = ["Overall (train full)", f"CV{FOLDS} mean (train)", "Validation (20%)"]
    # values = [overall_rmse, cv_mean, val_rmse]
    # plt.figure(figsize=(5,4))
    # plt.bar(labels, values)
    # plt.ylabel("RMSE")
    # plt.title("AMO — RMSE comparison")
    # plt.tight_layout()
    # plt.savefig("amo_rmse_comparison.png", dpi=150)
    # plt.show()

    # # Parity plot (validation)
    # plt.figure(figsize=(5,5))
    # plt.scatter(y_val_true, y_val_pred, s=28)
    # mn, mx = float(np.min(y_val_true)), float(np.max(y_val_true))
    # plt.plot([mn, mx], [mn, mx], linestyle="--")
    # plt.xlabel("True Eads (validation)")
    # plt.ylabel("Predicted Eads (validation)")
    # plt.title("AMO — Parity (validation)")
    # plt.tight_layout()
    # plt.savefig("amo_parity_validation.png", dpi=150)
    # plt.show()

if __name__ == "__main__":
    main()

Overwriting script.py


In [6]:
import pstats
import cProfile

cProfile.run('main()', 'stats')

p = pstats.Stats('stats')
p.strip_dirs().sort_stats("cumulative").print_stats()

************************************************ Starting Feature Space Construction in cpu ************************************************ 



************************************************ Starting 1 level of feature expansion...************************************************ 

************************************************ 1 Feature Expansion Completed with feature space size::: 1158 ************************************************ 

************************************************ Time taken to create the space is::: 0.0189516544342041  Seconds...************************************************ 

************************************************ Starting 2 level of feature expansion...************************************************ 

************************************************ 2 Feature Expansion Completed with feature space size::: 2770994 ************************************************ 

************************************************ Time taken to create the space i

In [15]:
!python -m cProfile -o profiling_results/model_opt script.py --config config.yaml

************************************************ Starting Feature Space Construction in cpu ************************************************ 



************************************************ Starting 1 level of feature expansion...************************************************ 

************************************************ 1 Feature Expansion Completed with feature space size::: 1158 ************************************************ 

************************************************ Time taken to create the space is::: 0.011970758438110352  Seconds...************************************************ 

************************************************ Starting 2 level of feature expansion...************************************************ 

************************************************ 2 Feature Expansion Completed with feature space size::: 2776779 ************************************************ 

************************************************ Time taken to create the space

In [16]:
import pstats

p = pstats.Stats('profiling_results/model_opt')
p.strip_dirs().sort_stats("cumulative").print_stats()

Wed Dec 17 20:56:53 2025    profiling_results/model_opt

         4106346 function calls (4039078 primitive calls) in 272.777 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.429    0.429  265.922  265.922 script.py:161(main)
       17    0.034    0.002  260.518   15.325 model.py:40(fit)
        3    1.927    0.642  200.990   66.997 script.py:115(cv_rmse_for_nterm)
       17    3.273    0.193  143.599    8.447 FeatureSpaceConstruction.py:548(feature_space)
       34   50.582    1.488  123.391    3.629 FeatureSpaceConstruction.py:374(combinations)
       17   12.696    0.747  107.905    6.347 Regressor.py:19(__init__)
       17   94.259    5.545   94.259    5.545 {built-in method torch.std_mean}
      341   33.492    0.098   33.492    0.098 {built-in method torch.cat}
       68   23.255    0.342   23.255    0.342 {built-in method torch.isinf}
      354    0.013    0.000   14.371    0.041 __init__.py:1(<module

In [9]:
p = pstats.Stats('profiling_results/comb_full_opt')
p.strip_dirs().sort_stats("cumulative").print_stats()

Wed Dec 17 19:03:42 2025    profiling_results/comb_full_opt

         4054962 function calls (3987007 primitive calls) in 288.445 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.128    0.128  256.872  256.872 script.py:154(main)
       17    1.416    0.083  254.047   14.944 model.py:87(fit)
        3    0.888    0.296  203.758   67.919 script.py:121(cv_rmse_for_nterm_on_train)
       17    2.446    0.144  125.798    7.400 FeatureSpaceConstruction.py:548(feature_space)
       34   52.331    1.539  112.626    3.313 FeatureSpaceConstruction.py:374(combinations)
       17   11.038    0.649  111.638    6.567 Regressor.py:25(__init__)
       17   91.400    5.376   91.400    5.376 {method 'std' of 'torch._C.TensorBase' objects}
      353    0.041    0.000   71.141    0.202 __init__.py:1(<module>)
      341   20.725    0.061   20.725    0.061 {built-in method torch.cat}
       68   16.613    0.244   16.613    0.244

In [20]:
p = pstats.Stats('profiling_results/initial')
p.strip_dirs().sort_stats("cumulative").print_stats()

Tue Dec 16 18:48:29 2025    profiling_results/initial

         115716099 function calls (115647857 primitive calls) in 607.251 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.111    0.111  599.279  599.279 script.py:154(main)
       17    1.580    0.093  589.691   34.688 model.py:87(fit)
        3    1.030    0.343  522.786  174.262 script.py:121(cv_rmse_for_nterm_on_train)
       17    2.008    0.118  466.872   27.463 FeatureSpaceConstruction.py:457(feature_space)
       34  255.614    7.518  452.489   13.308 FeatureSpaceConstruction.py:333(combinations)
       17   11.307    0.665  111.926    6.584 Regressor.py:25(__init__)
      545  107.023    0.196  107.023    0.196 {built-in method torch.cat}
       17   91.895    5.406   91.895    5.406 {method 'std' of 'torch._C.TensorBase' objects}
       68   18.531    0.273   18.531    0.273 {built-in method torch.isinf}
      136   18.267    0.134   18.267    0